In [ ]:
%pip install pandas
%pip install numpy 
%pip install matplotlib
%pip install seaborn

In [2]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
from collections import Counter

In [3]:
# Carregar cada CSV limpo
df_netflix = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_netflix_clean.csv')
df_hbo = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_hbo_clean.csv')
df_apple = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/df_apple_clean.csv')
df_prime = pd.read_csv('C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados_tratados/data_prime_video_clean.csv')


print("📂 DATASETS CARREGADOS:")
print(f"Netflix: {len(df_netflix):,} registros")
print(f"HBO Max: {len(df_hbo):,} registros") 
print(f"Apple TV: {len(df_apple):,} registros")
print(f"Prime Video: {len(df_prime):,} registros")

📂 DATASETS CARREGADOS:
Netflix: 21,391 registros
HBO Max: 9,776 registros
Apple TV: 18,692 registros
Prime Video: 72,943 registros


In [4]:
# Adicionar coluna de plataforma
df_netflix['platform'] = 'Netflix'
df_hbo['platform'] = 'HBO Max'
df_apple['platform'] = 'Apple TV+'
df_prime['platform'] = 'Prime Video'

In [5]:

# ==========================================
# UNINDO OS DATASETS
# ==========================================

platforms = {
    'Netflix': df_netflix,
    'HBO Max': df_hbo, 
    'Apple TV': df_apple,
    'Prime Video': df_prime
}
# Concatenar todos os DataFrames em um único DataFrame
df_streaming = pd.concat([df_netflix, df_hbo, df_apple, df_prime], ignore_index=True)


print(f"\n🎬 DATASET UNIFICADO CRIADO:")
print(f"Total de registros: {len(df_streaming):,}")
print(f"Total de colunas: {len(df_streaming.columns)}")
print(f"Colunas: {list(df_streaming.columns)}")



🎬 DATASET UNIFICADO CRIADO:
Total de registros: 122,802
Total de colunas: 8
Colunas: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes', 'platform']


In [ ]:
# Top 20 títulos mais votados 
top_20_votados = df_streaming.sort_values(by='imdbNumVotes', ascending=False).head(20)

# Exibir com formatação mais clara
print("TOP 20 TÍTULOS MAIS VOTADOS:")
print("-" * 80)
for i, row in top_20_votados.iterrows():
    print(f"{row.name + 1:2d}. {row['title']:<40} | {row['imdbNumVotes']:>10,} votos | {row['platform']}")


TOP 20 TÍTULOS MAIS VOTADOS:
--------------------------------------------------------------------------------
50004. The Shawshank Redemption                 |  3,046,940 votos | Prime Video
82. The Shawshank Redemption                 |  3,046,940 votos | Netflix
21463. The Shawshank Redemption                 |  3,046,940 votos | HBO Max
56. The Dark Knight                          |  3,023,583 votos | Netflix
21428. The Dark Knight                          |  3,023,583 votos | HBO Max
49930. The Dark Knight                          |  3,023,583 votos | Prime Video
22692. Inception                                |  2,685,476 votos | HBO Max
1862. Inception                                |  2,685,476 votos | Netflix
55216. Inception                                |  2,685,476 votos | Prime Video
50109. Fight Club                               |  2,467,361 votos | Prime Video
27934. Game of Thrones                          |  2,435,895 votos | HBO Max
49861. Forrest Gump               

Dá pra ver que The Shawshank Redemption é o mais votado no Prime, HBO e Netflix e The Dark Knight o segundo mais votado

In [7]:
# Verificar quantas plataformas cada título aparece
title_platforms = df_streaming.groupby('title')['platform'].apply(list).reset_index()
title_platforms['num_platforms'] = title_platforms['platform'].apply(len)

# Juntar com os dados de votos
top_with_distribution = df_streaming.merge(title_platforms, on='title')
top_final = top_with_distribution.sort_values(by='imdbNumVotes', ascending=False).drop_duplicates('title').head(20)

print("\nTOP 20 COM DISTRIBUIÇÃO POR PLATAFORMAS:")
print("-" * 100)
for i, row in top_final.iterrows():
    platforms_str = ', '.join(row['platform_y'])
    exclusivity = "EXCLUSIVO" if row['num_platforms'] == 1 else f"EM {row['num_platforms']} PLATAFORMAS"
    print(f"{row['title']:<40} | {row['imdbNumVotes']:>10,} votos | {exclusivity:<20} | {platforms_str}")



TOP 20 COM DISTRIBUIÇÃO POR PLATAFORMAS:
----------------------------------------------------------------------------------------------------
The Shawshank Redemption                 |  3,046,940 votos | EM 3 PLATAFORMAS     | Netflix, HBO Max, Prime Video
The Dark Knight                          |  3,023,583 votos | EM 4 PLATAFORMAS     | Netflix, HBO Max, Prime Video, Prime Video
Inception                                |  2,685,476 votos | EM 3 PLATAFORMAS     | Netflix, HBO Max, Prime Video
Fight Club                               |  2,467,361 votos | EM 2 PLATAFORMAS     | Prime Video, Prime Video
Game of Thrones                          |  2,435,895 votos | EXCLUSIVO            | HBO Max
Forrest Gump                             |  2,381,047 votos | EM 3 PLATAFORMAS     | Netflix, Apple TV+, Prime Video
Interstellar                             |  2,348,033 votos | EM 4 PLATAFORMAS     | Netflix, HBO Max, Apple TV+, Prime Video
Pulp Fiction                             |  2,336,631

In [9]:
# Criar ranking top 10 dos títulos mais votados com indicação das plataformas

# Agrupar plataformas por título
platforms_per_title = df_streaming.groupby('title')['platform'].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()

# Ordenar pelo número de votos e pegar top 10
top_10 = df_streaming.sort_values(by='imdbNumVotes', ascending=False).drop_duplicates('title').head(10)

# Juntar com as plataformas
ranking_top_10 = top_10[['title', 'imdbNumVotes']].merge(platforms_per_title, on='title')

# Formatar a saída para exibir o ranking
ranking_top_10 = ranking_top_10.reset_index(drop=True)
ranking_top_10.index += 1
ranking_top_10.columns = ['Título', 'Número de Votos', 'Plataformas Disponíveis']

print("\n🏆 RANKING DOS 10 PRINCIPAIS TITULOS ENTRE TODAS AS PLATAFORMAS:")
print(ranking_top_10.to_string(index=True))



🏆 RANKING DOS 10 PRINCIPAIS TITULOS ENTRE TODAS AS PLATAFORMAS:
                      Título  Número de Votos                   Plataformas Disponíveis
1   The Shawshank Redemption          3046940             HBO Max, Netflix, Prime Video
2            The Dark Knight          3023583             HBO Max, Netflix, Prime Video
3                  Inception          2685476             HBO Max, Netflix, Prime Video
4                 Fight Club          2467361                               Prime Video
5            Game of Thrones          2435895                                   HBO Max
6               Forrest Gump          2381047           Apple TV+, Netflix, Prime Video
7               Interstellar          2348033  Apple TV+, HBO Max, Netflix, Prime Video
8               Pulp Fiction          2336631  Apple TV+, HBO Max, Netflix, Prime Video
9               Breaking Bad          2334135                                   Netflix
10                The Matrix          2157238          